In [ ]:
import pyvista as pv
import os
import numpy as np
import matplotlib.pyplot as plt
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
print(pv.Report())

In [ ]:
vtu_files = [f  for f in os.listdir(os.path.join(os.curdir,'results')) if f.endswith('.vtu')]
vtu_files.sort()
print(f'{len(vtu_files)} vtu files found')

In [ ]:
mesh_t0 = pv.read(os.path.join(os.curdir,'results', vtu_files[0]))

In [ ]:
p = pv.Plotter()
p.add_mesh(mesh_t0, scalars="current density e", component=2, show_edges=False, cmap='jet')
p.add_mesh(mesh_t0.contour(isosurfaces=20, scalars='a', method='contour'),color='white',point_size=1, show_edges=False, opacity=0.5)
p.set_viewup([0,0,1])
p.camera.zoom('tight')
p.show()

In [ ]:
WIRE_1_ID = 1
WIRE_2_ID = 2

wire_1 = mesh_t0.threshold([WIRE_1_ID - 0.1, WIRE_1_ID + 0.1], scalars="GeometryIds")

In [ ]:
p = pv.Plotter()
p.add_mesh(wire_1, scalars="current density e", component=2, show_edges=True, cmap='jet')
p.set_viewup([0,1,0])

p.add_mesh(pv.Line((-1e-3,0,0), (1e-3,0,0)), color='green', line_width=5)
p.show()

In [ ]:
RESOLUTION=100
WIRE_DIAMETER = 2e-3

p = pv.Plotter(notebook=True, off_screen=True)
p.open_gif('j.gif')

mesh = pv.read(os.path.join(os.curdir,'results', vtu_files[0]))
wire_mesh = mesh.threshold([WIRE_1_ID - 0.1, WIRE_1_ID + 0.1], scalars="GeometryIds")
p.add_mesh(wire_mesh, scalars="current density e", component=2, show_edges=False, cmap='jet', lighting=False)
p.set_viewup([0,1,0])
p.camera.zoom('tight')
p.show()
p.update_scalar_bar_range([-4e5, 4e5])

wire_currrent = np.empty((len(vtu_files), 2))
wire_current_density = np.empty((len(vtu_files), RESOLUTION+1))
wire_positions = np.linspace(-WIRE_DIAMETER/2, WIRE_DIAMETER/2, RESOLUTION+1)

for idx, vtu_file in enumerate(vtu_files):
    logger.info(f'Processing {vtu_file}')
    mesh = pv.read(os.path.join(os.curdir,'results', vtu_file))
    for wire in [WIRE_1_ID, WIRE_2_ID]:
        wire_mesh = mesh.threshold([wire - 0.1, wire + 0.1], scalars="GeometryIds")
        wire_mesh_int = wire_mesh.integrate_data()
        wire_currrent[idx, wire-1] = wire_mesh_int['current density e'][0, 2] # 2 is the z component
        # sample across the wire,
        if wire == WIRE_1_ID:
            p.update_coordinates(wire_mesh.points, render=False)
            p.update_scalars(wire_mesh['current density e'][:,2], render=False)
            p.write_frame()
            line_sample = wire_mesh.sample_over_line((-WIRE_DIAMETER/2,0,0), (WIRE_DIAMETER/2,0,0), resolution=RESOLUTION)
            wire_current_density[idx] = line_sample['current density e'][:,2]

p.close()

In [ ]:
from matplotlib import pyplot as plt
plt.plot(wire_currrent[:,0], label='Wire 1')

In [ ]:
plt.plot(wire_positions/1e6, np.sqrt(np.mean(wire_current_density**2, axis=0)))


In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(sample_over_line["Distance"], sample_over_line["current density e"][:,2])

In [ ]:
sample_over_line["current density e"]